#Modules

In [1]:
import os
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf
import keras
LABLE_FILE = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/ppdata/subj01/behav/responses.tsv'
FMRI_DIR = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/GLM-fmri-data/subj01'
FMRI_DIR_STND = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/standardized-betas/subj01'
TRIAL_PER_SESS = 750
# !pip install line_profiler
# %load_ext line_profiler

Here we get the name of all files in the directory containgn the betas

In [2]:
  # returns a list of all files in the directory to be standardized
def getDirFiles(FMRI_DIR):
     files = [f for f in os.listdir(FMRI_DIR) if 
              os.path.isfile(os.path.join(FMRI_DIR, f)) and
              f[-5:] == '.hdf5']
     files.sort()                          
     return files

This shows each scan session is comprised of 750 scans. Each scan is of size (83, 104, 81)

In [54]:
files = getDirFiles(FMRI_DIR)
f = files[0]
print(f)
f = os.path.join(FMRI_DIR, f)
scan1=[]
with h5py.File(f, "r") as file:
    print(file.keys())
    print(file['betas'].shape)
    scan1 = np.array(file['betas'])
print(scan1.shape[0])

betas_session01.hdf5
<KeysViewHDF5 ['betas']>
(750, 83, 104, 81)
750


In [12]:
print(scan1.max())
print(scan1.min())
print(scan1.mean(0)[59,27,63])
mean = np.mean(scan1,axis=0)
print(mean[59,27,63])


32767
-32768
-80.90666666666667
-80.90666666666667


In [53]:
print('filtering labels...')
res = pd.read_csv(LABLE_FILE, sep='\t')
res = pd.DataFrame(res)
res = res.loc[res['SESSION'] <= 37]
res = res.loc[res['ISOLD'] == 1]
res = res.loc[res['ISCORRECT'] > -1]
res = res.filter(items = ['ISCORRECT'] )
res
# for index, row in res.iterrows():
#     print(index, row['ISCORRECT'])

filtering labels...


,ISCORRECT
45,1.0
68,1.0
90,1.0
107,1.0
121,1.0
...,...
27745,1.0
27746,1.0
27747,0.0
27748,1.0


#Standardize all betas in the directory

---



In [ ]:
path = os.path.join(FMRI_DIR_STND, FMRI_DIR_STND+"/standardized_betas.hdf5")
new_betas = h5py.File(path, 'w')
for index, row in res.iterrows():
    print(index, row['ISCORRECT'])
    
new_betas.close() 

In [8]:
class StandardizeDirectory:

  def __init__(self, loading_dir, dumping_dir, response_file):
    self.ld = loading_dir
    self.dd = dumping_dir
    self.rs = response_file
    self.indx = 0
    self.fmri_files = self.getDirFiles()
    self.file_handlers ={}
    self.open_files()

  def __del__(self):
    for value in self.file_handlers.items():
      value.close()
  # returns a list of all files in the directory to be standardized
  def getDirFiles(self):
     files = [f for f in os.listdir(self.ld) if 
              os.path.isfile(os.path.join(self.ld, f)) and
              f[-5:] == '.hdf5']
     files.sort()                          
     return files               

  def open_files(self): 
    for file_name in self.fmri_files:
      path = os.path.join(FMRI_DIR, file_name)
      self.file_handlers[file_name] = h5py.File(path, 'r')

  # Input: path of the responses.tsv file
  # Output: dictionary of indexes as keys with a corresponding class lable
  # in put the sesh number to filter , pass the session as an argument 
  def get_labels(self):
      print('filtering labels...')
      res = pd.read_csv(self.rs, sep='\t')
      res = pd.DataFrame(res)
      res = res.loc[res['SESSION'] <= 37]
      res = res.loc[res['ISOLD'] == 1]
      res = res.loc[res['ISCORRECT'] > -1]
      res = res.filter(items = ['ISCORRECT'] )
      return res

  def dump(self):
    path = os.path.join(self.dd, FMRI_DIR_STND+"/standardized_betas.hdf5")
    new_betas = h5py.File(path, 'w')
    new_betas.create_dataset()
    for session in self.fmri_files:
      partition = {} # ex: {'train': ['id-1', 'id-2', 'id-3'], 'validation': ['id-4']}
      labels = {} # ex: {'id-1': 0, 'id-2': 1, 'id-3': 2, 'id-4': 1}
      print('standardizing betas from '+ session)
      sesh = np.array(self.file_handlers[session]['betas'])
      mean = sesh.mean(0)
      std = sesh.std(0)
      standardized = (sesh-mean)/std
      standardized.size
      labels = self.get_labels()
      for index, row in labels:
        if index > self.indx:
          break
        new_betas

      
    
      group_dict = {'betas':[],'labels':[] }
      ready_data = self.standardize_data(sesh)
      
    
      
      
    new_betas.close()     
  
     


In [9]:
st = StandardizeDirectory(FMRI_DIR, FMRI_DIR_STND, LABLE_FILE)
st.dump()

begining dump
filtering labels...
...creating file 1....
DUMPED file 1
begining dump
filtering labels...
...creating file 2....
DUMPED file 2
begining dump
filtering labels...
...creating file 3....
DUMPED file 3
begining dump
filtering labels...
...creating file 4....
DUMPED file 4
begining dump
filtering labels...
...creating file 5....
DUMPED file 5
begining dump
filtering labels...
...creating file 6....
DUMPED file 6
begining dump
filtering labels...
...creating file 7....
DUMPED file 7
begining dump
filtering labels...
...creating file 8....
DUMPED file 8
begining dump
filtering labels...
...creating file 9....
DUMPED file 9
begining dump
filtering labels...
...creating file 10....
DUMPED file 10
begining dump
filtering labels...
...creating file 11....
DUMPED file 11
begining dump
filtering labels...
...creating file 12....
DUMPED file 12
begining dump
filtering labels...
...creating file 13....
DUMPED file 13
begining dump
filtering labels...
...creating file 14....
DUMPED file